In [1]:
import sys

import keras_tuner as kt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers

from src.cross_validator import KerasTunerAllChannelsCrossValidator
from src.dataset import load_dataset_train_val_all_channels, \
    load_dataset_train_test_all_channels
from src.models import mlp_builder as bare_model_builder
from src.network_utils import count_params

PWD = '../../..'
sys.path.append(PWD)

tf.get_logger().setLevel('ERROR')

In [2]:
PROJECT_NAME = 'mlp'

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_all/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_all/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 500
BATCH_SIZE = 8096
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_all, _, y_all, _ = load_dataset_train_test_all_channels(PWD)

X_all[(2, 11)].shape, y_all[(2, 11)].shape

((22134, 24), (22134,))

In [4]:
X_train, X_val, y_train, y_val = load_dataset_train_val_all_channels(PWD)

X_train = np.concatenate(list(X_train.values()))
y_train = np.concatenate(list(y_train.values()))
X_val = np.concatenate(list(X_val.values()))
y_val = np.concatenate(list(y_val.values()))

rng = np.random.default_rng(seed=42)

train_idx, val_idx = np.arange(len(X_train)), np.arange(len(X_val))
rng.shuffle(train_idx)
rng.shuffle(val_idx)

X_train, y_train = X_train[train_idx], y_train[train_idx]
X_val, y_val = X_val[val_idx], y_val[val_idx]

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((84824, 24), (84824,), (21208, 24), (21208,))

# Model

In [5]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=1, max_value=8, step=1, default=2)
    hp_units_mult = hp.Choice("units_mult", values=[1, 2, 4, 8, 16, 32], default=4)
    hp_unit_decrease_factor = hp.Choice("unit_decrease_factor", values=[1.0, 1.5, 2.0], default=1.5)
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_dropout = hp.Choice("dropout", values=[0.0, 0.2, 0.5])
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_n_hidden_layers, hp_units_mult, hp_unit_decrease_factor, hp_batch_normalization,
                               hp_input_batch_normalization, hp_dropout, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [6]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 18)                450       
                                                                 
 dense_1 (Dense)             (None, 12)                228       
                                                                 
 dense_2 (Dense)             (None, 1)                 13        
                                                                 
Total params: 691
Trainable params: 691
Non-trainable params: 0
_________________________________________________________________


In [7]:
model_callbacks = [callbacks.EarlyStopping(patience=30, min_delta=ES_MIN_DELTA),
                   callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=5)]

# Bayesian tuner

In [8]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

Trial 40 Complete [00h 00m 20s]
val_loss: 8.982244968414307

Best val_loss So Far: 8.692980766296387
Total elapsed time: 01h 11m 37s


# Results

In [9]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_hidden_layers': 4, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 30529
========== Model 1 ==========
{'n_hidden_layers': 4, 'units_mult': 16, 'unit_decrease_factor': 2.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 106849
========== Model 2 ==========
{'n_hidden_layers': 6, 'units_mult': 32, 'unit_decrease_factor': 2.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 6389473
========== Model 3 ==========
{'n_hidden_layers': 8, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 70849
========== Model 4 ==========
{'n_hidden_layers': 3, 'u

# Cross-validation for top 5 models

In [10]:
cross_validator = KerasTunerAllChannelsCrossValidator(bayesian_tuner, list(X_all.values()), list(y_all.values()),
                                                      model_builder, directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                                      n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                                      es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV,
                                                      n_executions=CROSSVAL_N_EXEC, overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_hidden_layers': 4, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 30529
Got score: 8.5377 (8.4940, 8.5814)
Got score: 8.2280 (8.2266, 8.2295)
Got score: 8.3074 (8.3236, 8.2912)
Got score: 8.1867 (8.1543, 8.2192)
Got score: 8.5293 (8.6464, 8.4121)


{'n_hidden_layers': 4, 'units_mult': 16, 'unit_decrease_factor': 2.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0, 'normalize_signal': False}
Number of parameters: 106849
Got score: 8.5930 (8.6659, 8.5202)
Got score: 8.3014 (8.2938, 8.3090)
Got score: 8.4406 (8.4394, 8.4417)
Got score: 8.3182 (8.3333, 8.3032)
Got score: 8.5096 (8.5282, 8.4909)


{'n_hidden_layers': 6, 'units_mult': 32, 'unit_decrease_factor': 2.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 6389473
Got score: 8.5191 (8.6252, 8.4129)
Got score: 8.1838 (8.2583, 8.1093)
Got score: 8.3762 (8.4190, 8.3334)
Got score: 8.2816 (8.2061, 8.3571)
Got score: 8.5444 (8.5363, 8.5524)


{'n_hidden_layers': 8, 'units_mult': 32, 'unit_decrease_factor': 1.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 70849
Got score: 8.7599 (8.7255, 8.7943)
Got score: 8.4413 (8.3315, 8.5511)
Got score: 8.5920 (8.5119, 8.6720)
Got score: 8.3911 (8.4006, 8.3817)
Got score: 8.8608 (9.0433, 8.6783)


{'n_hidden_layers': 3, 'units_mult': 32, 'unit_decrease_factor': 1.5, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2, 'normalize_signal': False}
Number of parameters: 52585
Got score: 8.6880 (8.6934, 8.6826)
Got score: 8.4591 (8.5512, 8.3670)
Got score: 8.6220 (8.7078, 8.5362)
Got score: 8.3411 (8.3365, 8.3457)
Got score: 8.7141 (8.7704, 8.6577)


In [11]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.36,0.15,30529
1,8.43,0.11,106849
2,8.38,0.14,6389473
3,8.61,0.18,70849
4,8.56,0.14,52585
